In [167]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

In [148]:
# Chrome Driver 104
DRIVER_PATH = '/Users/edwardhuang/Documents/interview/Dave And Busters/chromedriver'

# Use Chrome Beta, compatible with above Chrome Driver
options = Options()
#options.headless = True
options.binary_location = '/Applications/Google Chrome Beta.app/Contents/MacOS/Google Chrome Beta'

In [163]:
# Create array of D&Bs Locations in the format of Column x State x Location
# This is unideal because it hardcodes the structure of the store location page
# Ideally you would get the parent and then loop through each child but lets get a working prototype before coming back and making the selenium call more advance.
allLocs = [[[1,2],[1]],[[],[]],[[],[1,2,3]],[[],[]]]
address1List = []
address2List = []
address3List = []


n = 1

for col in range(len(allLocs)):
    for state in range(len(allLocs[col])):
        for location in allLocs[col][state]:
            # Using XPATH since it fits well with the column/state/location structure of page
            if location is not None:
                xpath = f"//*[@id='dnb-locations-list']/div[2]/div[{col+1}]/div[{state+1}]/ul/li[{location}]/a"   
            else:
                xpath = f"//*[@id='dnb-locations-list']/div[2]/div[{col+1}]/div[{state+1}]/ul/li/a"   
            
            print(xpath)
            
            # Is it messy to restart driver every time for a new location?
            driver = webdriver.Chrome(options=options,executable_path=DRIVER_PATH)
            driver.get('https://www.daveandbusters.com/locations')
            wait = WebDriverWait(driver, 30)  # Wait for page to load (uper bound)

            ActionChains(
                driver).move_to_element(  #the link is not clickable immediately, so you need to scroll down to it.
                    wait.until(
                        EC.element_to_be_clickable(
                            (By.XPATH, xpath)
                        )
                    )
                ).click(
            ).perform()
            
            time.sleep(7) # Wait for store page to load. This is probably bad practice, you can maybe add this to action chains and use wait.until again?
            address1 = driver.find_element(
                By.XPATH, "//*[@id='location-details-widget']/div/div[1]/div[2]/div[1]/div/div[2]").text
            address2 = driver.find_element(
                By.XPATH, "//*[@id='location-details-widget']/div/div[1]/div[2]/div[1]/div/div[3]").text
            address3 = driver.find_element(
                By.XPATH, "//*[@id='location-details-widget']/div/div[1]/div[2]/div[1]/div/div[4]").text

            address1List.append(address1)
            address2List.append(address2)
            address3List.append(address3)
            
            print(n)
            n += 1
            
driver.close()

//*[@id='dnb-locations-list']/div[2]/div[1]/div[1]/ul/li[1]/a


<ipython-input-163-e213be66a36e>:26: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options,executable_path=DRIVER_PATH)


check2
1
//*[@id='dnb-locations-list']/div[2]/div[1]/div[1]/ul/li[2]/a
check2
2
//*[@id='dnb-locations-list']/div[2]/div[1]/div[2]/ul/li[1]/a
check2
3
//*[@id='dnb-locations-list']/div[2]/div[3]/div[2]/ul/li[1]/a
check2
4
//*[@id='dnb-locations-list']/div[2]/div[3]/div[2]/ul/li[2]/a
check2
5
//*[@id='dnb-locations-list']/div[2]/div[3]/div[2]/ul/li[3]/a
check2
6


In [169]:
df = pd.DataFrame(data = {
    'Address 1': address1List,
    'Address 2': address2List,
    'City State Zip': address3List
})

df['Full Address'] = df['Address 1'] + ' ' + df['Address 2'] + ' ' + df['City State Zip']

df

In [195]:
# You can make this a function and appply it to the df

n = 1
reviewList = []

for store in df['Full Address']:    
    driver = webdriver.Chrome(options=options,executable_path=DRIVER_PATH)
    driver.get('https://www.google.com')

    # Identify search box
    m = driver.find_element(By.NAME,"q")
    # Enter search text
    m.send_keys('Dave and Busters ' + store)
    time.sleep(0.2)
    # Perform Google search
    m.send_keys(Keys.ENTER)

    time.sleep(7) # Bad practice? use wait.until?

    # I used class name this time because it seems more generic than xpath. 
    # There might be an even better search element.
    review = driver.find_element(
        By.CLASS_NAME,'Aq14fc').text
    
    reviewList.append(review)
    print(review)
            
driver.close()

<ipython-input-195-e85013b12bdf>:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options,executable_path=DRIVER_PATH)


4.1
4.3
4.0
4.3
4.1
4.1


In [196]:
df['Reviews'] = reviewList
df

,Address 1,Address 2,City State Zip,Full Address,Reviews
0,2700 Riverchase Galleria,Ste 110,"Birmingham, AL, 35244","2700 Riverchase Galleria Ste 110 Birmingham, A...",4.1
1,950 Makers Way NW,,"Huntsville, AL, 35806","950 Makers Way NW Huntsville, AL, 35806",4.3
2,800 East Dimond Blvd.,Suite 240,"Anchorage, AK, 99515","800 East Dimond Blvd. Suite 240 Anchorage, AK,...",4.0
3,200 Premium Outlets Dr,,"Blackwood, NJ, 08012","200 Premium Outlets Dr Blackwood, NJ, 08012",4.3
4,310 Willowbrook Mall,,"Wayne, NJ, 07470","310 Willowbrook Mall Wayne, NJ, 07470",4.1
5,274 Woodbridge Center Dr,,"Woodbridge, NJ, 07095","274 Woodbridge Center Dr Woodbridge, NJ, 07095",4.1


In [204]:
test = [1,2,3]
test[1:]

[2, 3]